# Automated ML

Importing Depedencies required for the project

In [1]:
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

## Dataset

### Overview

The Dataset used is "Stroke prediction Dataset"

According to the World Health Organization (WHO) stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths. This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.

Attribute Information

1) id: unique identifier

2) gender: "Male", "Female" or "Other"

3) age: age of the patient

4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension

5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease

6) ever_married: "No" or "Yes"

7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"

8) Residence_type: "Rural" or "Urban"

9) avg_glucose_level: average glucose level in blood

10) bmi: body mass index

11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*

12) stroke: 1 if the patient had a stroke or 0 if not

In this project, Azure AutoML will be used to make prediction on the death event based on patient's 10clinical features.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoML-run'

experiment=Experiment(ws, experiment_name)

In [3]:
dataset = Dataset.get_by_name(ws, 'stroke-prediction')

In [4]:
dataset.take(5).to_pandas_dataframe()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,True,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,True,Self-employed,Rural,202.21,N/A,never smoked,1
2,31112,Male,80.0,0,1,True,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,True,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,True,Self-employed,Rural,174.12,24,never smoked,1


In [5]:
data_train, data_test = dataset.random_split(0.9)

## Configuring Compute Cluster 

In [6]:
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration



In [7]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=data_train,
                             label_column_name="stroke", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_fb8bc4fa-774c-438c-a30c-e39b66933613

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+------------

## Run Details


In [9]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-run,AutoML_fb8bc4fa-774c-438c-a30c-e39b66933613,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
RunDetails(remote_run).show

<bound method _WidgetRunDetailsBase.show of <azureml.widgets._automl._run_details._AutoMLRunDetails object at 0x7ff5685cef98>>

In [11]:
remote_run.wait_for_completion()

{'runId': 'AutoML_fb8bc4fa-774c-438c-a30c-e39b66933613',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-13T14:56:19.770911Z',
 'endTimeUtc': '2021-02-13T15:20:37.285579Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-run","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-138804","workspace_name":"quick-starts-ws-138804","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"feat

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [13]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
AutoML-run,AutoML_fb8bc4fa-774c-438c-a30c-e39b66933613_37,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
fitted_model


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('stackensembleclassifier',
                 StackE...
                                         meta_learner=LogisticRegressionCV(Cs=10,
                                                                           class_weight=None,
                                                                           cv=None,
                                                                           dual=False,
                           

In [15]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(fitted_model._final_estimator)
print(best_run.get_tags())

Best Run Id:  AutoML_fb8bc4fa-774c-438c-a30c-e39b66933613_37

 Accuracy: 0.9512372415705647
StackEnsembleClassifier(base_learners=[('31',
                                        Pipeline(memory=None,
                                                 steps=[('maxabsscaler',
                                                         MaxAbsScaler(copy=True)),
                                                        ('lightgbmclassifier',
                                                         LightGBMClassifier(boosting_type='goss',
                                                                            class_weight=None,
                                                                            colsample_bytree=0.5944444444444444,
                                                                            importance_type='split',
                                                                            learning_rate=0.04211105263157895,
                                               

In [16]:
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

model_name = best_run.properties['model_name']
model_name

'AutoMLfb8bc4fa737'

In [25]:
from azureml.automl.core.shared import constants 
env = best_run.get_environment()

script_file = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [26]:
#Register the fitted model
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML model')

In [30]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                memory_gb = 1, 
                                                enable_app_insights = True,
                                                auth_enabled = True,
                                               description = 'Predict whether person had stroke or not')
                                            

aci_service_name = 'stroke-prediction2'
print(aci_service_name)

stroke-prediction2


In [31]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
State: Healthy
Scoring URI: http://04580459-2f0a-475f-93e9-718991121d05.southcentralus.azurecontainer.io/score


In [32]:
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)
print("Keys: " + service.get_keys()[0])
print("Swagger URI: " + service.swagger_uri)

State: Healthy
Scoring URI: http://04580459-2f0a-475f-93e9-718991121d05.southcentralus.azurecontainer.io/score
Keys: 9qcgCTJk82rgEpHadVyOjwiGN9FsYmGY
Swagger URI: http://04580459-2f0a-475f-93e9-718991121d05.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [45]:

%run endpoint.py

{"result": [0, 0]}


In [48]:
data_test = data_test.dropna()
data_sample = data_test.sample(3)
y_true = data_sample.pop('stroke')
sample_json = json.dumps({'data':data_sample.to_dict(orient='records')})
print(sample_json)

{"data": [{"id": 47585, "gender": "Female", "age": 31.0, "hypertension": 0, "heart_disease": 0, "ever_married": false, "work_type": "Self-employed", "Residence_type": "Urban", "avg_glucose_level": 62.68, "bmi": "35.8", "smoking_status": "never smoked"}, {"id": 32257, "gender": "Female", "age": 47.0, "hypertension": 0, "heart_disease": 0, "ever_married": true, "work_type": "Private", "Residence_type": "Urban", "avg_glucose_level": 210.95, "bmi": "50.1", "smoking_status": "Unknown"}, {"id": 24246, "gender": "Male", "age": 7.0, "hypertension": 0, "heart_disease": 0, "ever_married": false, "work_type": "children", "Residence_type": "Urban", "avg_glucose_level": 77.76, "bmi": "18.1", "smoking_status": "Unknown"}]}


In [49]:
output = service.run(sample_json)
print('Prediction: ', output)
print('True Values: ', y_true.values)

Prediction:  {"result": [0, 0, 0]}
True Values:  [0 0 0]


In [50]:
service.get_logs(())

'2021-02-13T16:09:38,172266233+00:00 - gunicorn/run \n2021-02-13T16:09:38,173041585+00:00 - iot-server/run \n2021-02-13T16:09:38,173909543+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-02-13T16:09:38,182736431+00:00 - rsyslog/run 

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()